# Target Audience for Direct Marketing in Starbucks Rewards Mobile App

This is part 1 of Starbucks capstone project. In this part, I clean the data, merge datasets, perform necessary checks and aggregate.

See `2_Starbucks_eda.ipynb`, `3_Starbucks_modeling.ipynb` and `4_Starbucks_modeling.ipynb` for more information on other steps.

## 1.1. Project Overview

In this project, I analyze the customer behavior in the Starbucks rewards mobile app. After signing up for the app, customers receive promotions every few days. The task is to identify what customers are influenced by promotional offers the most and what types of offers to send them in order to maximize the revenue. 

There are three types of promotions:
  * discount 
  * bogo (buy one, get one free)
  * informational - product advertisiment without any price off

Each offer is valid for certain number of days before it expires. Discounts and bogos have also different difficulty level, depending on how much the customer needs to spend in order to earn the promotion. Promotions are distributed via different multiple channels (social, web, email, mobile).

All transactions made through the app are tracked automatically. The app also records information about which offers have been sent, which have been viewed and which have been completed and when these three events happened.

## 1.2. Dataset Overview
The data is organized in three files:

* `portfolio.json` (10 offers x 6 fields) - offer types sent during 30-day test period 
* `profile.json` (17000 users x 5 fields) - demographic profile of app users 
* `transcript.json` (306648 events x 4 fields) - event log on transactions, offers received, offers viewed, and offers completed 

Here is the schema and explanation of each variable in the files:

**portfolio.json**

  * id (string) - offer id
  * offer_type (string) - type of offer ie BOGO, discount, informational
  * difficulty (int) - minimum required spend to complete an offer
  * reward (int) - reward given for completing an offer
  * duration (int) - time for offer to be open, in days
  * channels (list of strings)

**profile.json**
  * age (int) - age of the customer 
  * became_member_on (int) - date when customer created an app account
  * gender (str) - gender of the customer 
  * id (str) - customer id
  * income (float) - customer's income

**transcript.json**
  * event (str) - record description (ie transaction, offer received, offer viewed, etc.)
  * person (str) - customer id
  * time (int) - time in hours since start of test. The data begins at time t=0
  * value - (dict of strings) - either an offer id or transaction amount depending on the record


## 1.3. Problem Statement

The aim of this project is to identify target audience for a successful marketing campaign in direct marketing:

> Direct marketing describes the marketer's efforts to directly reach the customer through direct marketing communications (direct mail, e-mail, social media, and similar "personalized" or one-on-one means). The direct marketiing effort requires marketers to have a target list of customers that will each receive a marketing message tailored to their needs and interests. --- John A. Davis: "Measuring Marketing", 3d Edition, 2018, Part 8.

To solve this task, I decided to use customer segmentation using **K-means clustering technique**. 

The idea is to divide app users into major groups - those more prone to discounts vs. those more keen on bogos vs. those that are not interested in promotions at all. The number of groups was decided at the later stage depending on the actual patterns in the final dataset.

## 1.4. Metrics

The critical decision in customer segmentation task is to choose the optimal number of segments. The problem is that unsupervised machine learning doesn't have clearly defined benchmark metrics for model performance evaluation on par with supervised ML (e.g. accuracy score, f1-score, AUC, etc.). Instead there is a number of heuristics that aid analysts during decision-making process, but which ultimately doesn't say anything whether the modeling results are good or bad. For k-means clustering, these heuristics are elbow curve method and silouhette score for deciding upon optimal number of clusters (see section 4.4. for more details). 

To evaluate the segmentation results, I relied on the following marketing metrics:
  * **Response Rate (RR)** - the percentage of customers who viewed an offer relative to the number of customers that received the offer
  * **Conversion Rate (CVR)** - the percentage of customers who completed an offer relative to the number of customers who viewed an offer

These two marketing metrics helps marketers to improve efficiency and reduce costs of marketing campaign. The response rate tells how many customers are interested in offers, while the conversion rates shows whether the offers sent were attractive enough to complete them.

By calculating each customer's RR and CVR for all bogos, discounts and informational offers sent to him/her, I was able to evaluate the resulting segments in alignment with the project's task, i.e. identify target audience for each promotion type. 

In [1]:
import time
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from pandas.io.json import json_normalize

%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Data Exploration

## 1.1. Portfolio

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


OFFER TYPES: 
- 2 Informational offers - provide no rewards, last either 3 (+social channel) or 4 days (+web channel). 
- 4 BOGO types (difficulty=reward) - 2 lasting for 5 days (one easier - \\$5, one more difficult - \\$10) and 2 lasting for 7 days (one easier - \\$5, one more difficult - \\$10) 
- 4 discounts - 2 lasting for 7 days (one easier - \\$7d / \\$3r, one more difficult - \\$10d / \\$2r) + 2 lasting for 10 days (one easier - \\$10d / \\$2r, one more difficult - \\$20d / \\$5r). Both difficult versions of duration types differ in channels - shorter--> no social, longer--> no social+no mobile). So the longest, the most difficult offer is distributed only via web/email. Difficulty correlates with instancy of channels.

In [3]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
reward        10 non-null int64
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
offer_type    10 non-null object
id            10 non-null object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


## 1.2. Profile

In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
if (profile.id.value_counts() > 1).sum() == 0:
    print("profile's ids are all unique")

profile's ids are all unique


In [6]:
print(f"there are {profile.shape[0]} customers in profile")

there are 17000 customers in profile


In [7]:
#Change type of 'became_member_on' column from int to time:
profile_clean = profile.copy()
profile_clean.became_member_on = pd.to_datetime(profile_clean.became_member_on.astype(str), format="%Y/%m/%d")

In [8]:
# renaming 'id' column to 'person' in profile dataset so that we can later merge it with transcript dataset
profile_clean.rename(columns={profile_clean.columns[2]:'person'}, inplace=True)

Reencoding missing values in Profile with NaNs:

In [9]:
# Missing values in 'gender' column are recorded as NoneType, in 'income' column - as NaNs. 
type(profile_clean.gender[0]), type(profile_clean.income[0])

(NoneType, numpy.float64)

In [10]:
# Replace NoneType with Nans in gender column
profile_clean.gender.fillna(value=np.nan, inplace=True)

In [11]:
#checking the result of transformation
type(profile_clean.gender[0]), type(profile.income[0])

(float, numpy.float64)

In [12]:
# Replace 118 with Nans in age column:
profile_clean.age.replace(118, np.nan, inplace=True)

In [13]:
#reorganize the columns order
profile_clean = profile_clean[['person', 'became_member_on', 'gender', 'age', 'income']]

In [14]:
profile_clean.head()

,person,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55.0,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,NaN,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75.0,100000.0
4,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,NaN,NaN


## 1.3 Transcript

In [15]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [16]:
transcript.shape

(306534, 4)

In [17]:
print(f"experiment lasted for max {transcript.time.max()/24} days")

experiment lasted for max 29.75 days


Let's see the typical transaction record for one person:

In [18]:
transcript[transcript.person == '78afa995795e4d85b5d9ceeca43f5fef']

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6
47582,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132
49502,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216
87134,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.67},222
92104,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 29.72},240
141566,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 23.93},378


Note: We have many records for one person that are organized chronologically. 'Value' column is currently a json format with either 'offer_id' for offers completed, 'amount' of transactions and 'offer id' for offers received and offers viewed.

In [19]:
# normalize json in value column
value_normalized = json_normalize(transcript['value'])

In [20]:
value_normalized.head()

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [21]:
# merging the two columns 'offer id' and 'offer_id' into new one 'id'
value_normalized['id'] = value_normalized['offer id'].fillna(value_normalized['offer_id'])

In [22]:
# dropping the redundant columns 'offer id', 'offer_id'
value_normalized = value_normalized.drop(['offer id', 'offer_id'], axis=1)

In [23]:
#rename the column 'reward' to 'rewarded'
value_normalized.rename(columns={value_normalized.columns[1]:'rewarded'}, inplace=True)

In [24]:
# join value_normalized with transcript 
transcript_clean = transcript.join(value_normalized)

In [25]:
# dropping 'value', 'event' columns
transcript_clean = transcript_clean.drop(['value'], axis=1)

In [26]:
# reorganize the order of columns
transcript_clean = transcript_clean[['person', 'id', 'event', 'amount', 'rewarded', 'time']]

In [27]:
transcript_clean.head()

,person,id,event,amount,rewarded,time
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,NaN,NaN,0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,NaN,NaN,0
2,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,offer received,NaN,NaN,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,offer received,NaN,NaN,0
4,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,NaN,NaN,0


## Combining datasets

1.1. Merging Profile and Transcript datasets on customer ids:

In [28]:
# checking the number of rows in the datasets
profile_clean.shape, transcript.shape

((17000, 5), (306534, 4))

In [29]:
# because transcipt has more rows, we will do the right join
df = profile_clean.merge(transcript_clean, on='person', how='right')

In [30]:
df.shape

(306534, 10)

1.2. Merging with Portfolio dataset

In [31]:
starbucks = df.merge(portfolio, on='id', how='left')

In [32]:
starbucks.head()

,person,became_member_on,gender,age,income,id,event,amount,rewarded,time,reward,channels,difficulty,duration,offer_type
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,offer received,NaN,NaN,168,2.0,"[web, email, mobile]",10.0,7.0,discount
1,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,offer viewed,NaN,NaN,216,2.0,"[web, email, mobile]",10.0,7.0,discount
2,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,NaN,NaN,336,5.0,"[web, email]",20.0,10.0,discount
3,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,NaN,NaN,348,5.0,"[web, email]",20.0,10.0,discount
4,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,NaN,transaction,0.35,NaN,360,NaN,NaN,NaN,NaN,NaN


In [33]:
starbucks.shape

(306534, 15)

One-Hot Encode offer type

In [34]:
starbucks_fin = pd.get_dummies(starbucks, columns=['event', 'offer_type'])

In [35]:
# reordering the columns
starbucks_fin = starbucks_fin[['person', 'became_member_on', 'age', 'income','gender',
                               'id', 'amount','rewarded', 'time', 
                               'event_offer completed', 'event_offer received', 'event_offer viewed',
                               'event_transaction', 'offer_type_bogo', 'offer_type_discount',
                               'offer_type_informational']]

In [36]:
# renaming the columns
starbucks_fin.columns = ['person', 'became_member_on', 'age', 'income', 'gender',
                         'id', 'amount','rewarded', 'time',
                         'offer_completed', 'offer_received', 'offer_viewed',
                         'transaction', 'bogo', 'discount', 'informational']

In [37]:
starbucks_fin.head()

,person,became_member_on,age,income,gender,id,amount,rewarded,time,offer_completed,offer_received,offer_viewed,transaction,bogo,discount,informational
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,168,0,1,0,0,0,1,0
1,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,216,0,0,1,0,0,1,0
2,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,336,0,1,0,0,0,1,0
3,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,348,0,0,1,0,0,1,0
4,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,NaN,0.35,NaN,360,0,0,0,1,0,0,0


In [38]:
starbucks_fin.shape

(306534, 16)

**Let's summarize the above steps in functions:**

In [39]:
def clean_profile_dataset(profile):
    '''
    Input:
        profile: original profile dataset 
    Output:
        profile_clean: cleaned profile dataset
    '''
    profile_clean = profile.copy()  
    # renaming 'id' column to 'person' in profile dataset 
    profile_clean.rename(columns={profile_clean.columns[2]:'person'}, inplace=True)
    #Change type of 'became_member_on' column from int to time:
    profile_clean.became_member_on = pd.to_datetime(profile_clean.became_member_on.astype(str), format="%Y/%m/%d")
    # Replace NoneType with Nans in gender column
    profile_clean.gender.fillna(value=np.nan, inplace=True)
    # Replace 118 with Nans in age column:
    profile_clean.age.replace(118, np.nan, inplace=True)
    
    return profile_clean

In [40]:
def normalize_json_in_transcript_dataset(transcript):
    '''
    Input:
        transcript: original transcript dataset 
    Output:
        transcript_clean: normalized transcript dataset
    '''    
    # normalize json in value column
    value_normalized = json_normalize(transcript['value'])
    # merging the two columns 'offer id' and 'offer_id' into new one 'id'
    value_normalized['id'] = value_normalized['offer id'].fillna(value_normalized['offer_id'])
    # dropping the redundant columns 'offer id', 'offer_id'
    value_normalized = value_normalized.drop(['offer id', 'offer_id'], axis=1)
    #rename the column 'reward' to 'rewarded'
    value_normalized.rename(columns={value_normalized.columns[1]:'rewarded'}, inplace=True)
    # join value_normalized with transcript 
    transcript_clean = transcript.join(value_normalized)    
    # dropping 'value', 'event' columns
    transcript_clean = transcript_clean.drop(['value'], axis=1)
    
    return transcript_clean

In [41]:
def merge_cleaned_datasets(profile, transcript, portfolio):
    '''
    Input:
        profile: original profile dataset
        transcript: original transcript dataset
        portfolio: original portfolio dataset
    Output:
        starbucks: merged dataset with selected columns
    
    '''
    profile_clean = clean_profile_dataset(profile)
    transcript_clean = normalize_json_in_transcript_dataset(transcript)
    
    #combine datasets
    df = profile_clean.merge(transcript_clean, on='person', how='right')
    starbucks = df.merge(portfolio, on='id', how='left')
    
    #one-hot encode 'event' and 'offer_type'
    starbucks = pd.get_dummies(starbucks, columns=['event', 'offer_type'])
    
    # reordering the columns (exclude 'channels', 'difficulty')
    starbucks = starbucks[['person', 'became_member_on', 'age', 'gender', 'income',
                           'time', 'id', 'duration', 'reward', 
                           'amount','rewarded', 
                           'event_offer completed', 'event_offer received', 'event_offer viewed',
                           'event_transaction', 'offer_type_bogo', 'offer_type_discount',
                           'offer_type_informational']]
    
    # renaming the columns
    starbucks.columns = ['person', 'became_member_on', 'age', 'gender', 'income', 
                         'time', 'offer_id', 'offer_duration', 'offer_reward',
                         'amount','rewarded', 'offer_completed', 'offer_received', 'offer_viewed',
                         'transaction', 'bogo', 'discount', 'informational']
    return starbucks

In [42]:
starbucks = merge_cleaned_datasets(profile, transcript, portfolio)

In [43]:
starbucks.head()

,person,became_member_on,age,gender,income,time,offer_id,offer_duration,offer_reward,amount,rewarded,offer_completed,offer_received,offer_viewed,transaction,bogo,discount,informational
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,168,2906b810c7d4411798c6938adc9daaa5,7.0,2.0,NaN,NaN,0,1,0,0,0,1,0
1,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,216,2906b810c7d4411798c6938adc9daaa5,7.0,2.0,NaN,NaN,0,0,1,0,0,1,0
2,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,5.0,NaN,NaN,0,1,0,0,0,1,0
3,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,348,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,5.0,NaN,NaN,0,0,1,0,0,1,0
4,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,360,NaN,NaN,NaN,0.35,NaN,0,0,0,1,0,0,0


## Aggregation at Offer Level:

To correctly attribute conversion rates and response rate, we need to organize data by time. In the following, I create columns with time for when offers were received, when they were viewed and when they were completed. 

In [44]:
starbucks_time = starbucks[starbucks.transaction == 0][['person', 'offer_id', 'offer_received', 'offer_viewed', 'offer_completed', 'time']]

In [45]:
starbucks_time.head()

,person,offer_id,offer_received,offer_viewed,offer_completed,time
0,68be06ca386d4c31939f3a4f0e3dd783,2906b810c7d4411798c6938adc9daaa5,1,0,0,168
1,68be06ca386d4c31939f3a4f0e3dd783,2906b810c7d4411798c6938adc9daaa5,0,1,0,216
2,68be06ca386d4c31939f3a4f0e3dd783,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,336
3,68be06ca386d4c31939f3a4f0e3dd783,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,348
5,68be06ca386d4c31939f3a4f0e3dd783,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,408


In [46]:
# there are offers that were sent the same person several times:
starbucks_time[(starbucks_time['offer_received'] == 1)
               & (starbucks_time['offer_id'] == 'fafdcd668e3743c1bb461111dcafc2a4')
              & (starbucks_time['person'] == '0020c2b971eb4e9188eac86d93036a77')]

,person,offer_id,offer_received,offer_viewed,offer_completed,time
46131,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,0
46139,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,336


In [47]:
print("total number of offers sent:", starbucks_time[starbucks_time['offer_received'] == 1].shape[0])

total number of offers sent: 76277


In [48]:
starbucks_time_aggr = starbucks_time.groupby(['person', 'offer_id']).sum().reset_index()
print("number of offers received only once:", starbucks_time_aggr[starbucks_time_aggr.offer_received == 1].offer_received.sum())
print("number of offers that were received more than once:", starbucks_time_aggr[starbucks_time_aggr.offer_received > 1].offer_received.sum())

number of offers received only once: 51570
number of offers that were received more than once: 24707


Later in this notebook, we will produce a lot of values with 0s after aggregation steps. In time column 0 means hour, but when aggregating things 0 means no event. So to avoid confusion, let's reencode time=0 with time=0.5.

In [49]:
starbucks_time['time'] = starbucks_time.time.replace(0, 0.5)

Because we have offers that were sent more than once, the offer_id is no longer unique identifier. We need to create a new unique id of each offer sent, irrespective whether customer already received it earlier or not.
This is not a trivial task. To do this, I use the function cumcount() to create a counter for identical offers. This is possible because our data is time ordered.

In [50]:
starbucks_time['id_unique_received'] = starbucks_time[starbucks_time.offer_received == 1].groupby(['person', 'offer_id']).cumcount()
starbucks_time['id_unique_viewed'] = starbucks_time[starbucks_time.offer_viewed == 1].groupby(['person', 'offer_id']).cumcount()
starbucks_time['id_unique_completed'] = starbucks_time[starbucks_time.offer_completed == 1].groupby(['person', 'offer_id']).cumcount()
starbucks_time['id_unique_events'] = starbucks_time[['id_unique_received', 'id_unique_viewed', 'id_unique_completed']].max(axis=1).values
starbucks_time['id_unique'] = starbucks_time['offer_id'] + "-" + starbucks_time['id_unique_events'].apply(lambda x: str(x))

In [51]:
starbucks_time['offer_received_time'] = starbucks_time['offer_received']*starbucks_time.time
starbucks_time['offer_viewed_time'] = starbucks_time['offer_viewed']*starbucks_time.time
starbucks_time['offer_completed_time'] = starbucks_time['offer_completed']*starbucks_time.time
starbucks_time = starbucks_time[['person', 'id_unique', 'offer_id', 'time', 'offer_received_time', 'offer_viewed_time', 'offer_completed_time']]

In [52]:
# unstack values to get to the level of each (person, offer id) tuple
# take max values to avoid 0s
starbucks_time_full = starbucks_time.groupby(by=['person', 'id_unique', 'time']).max().unstack()
starbucks_time_full.fillna(0, inplace=True)

In [53]:
# create our final time-based dataset for each offer sent
offers = pd.DataFrame(starbucks_time_full.index.get_level_values('id_unique'), starbucks_time_full.index.get_level_values('person')).reset_index()

offers['offer_received_time'] = starbucks_time_full['offer_received_time'].values.max(axis=1)
offers['offer_viewed_time'] = starbucks_time_full['offer_viewed_time'].values.max(axis=1)
offers['offer_completed_time'] = starbucks_time_full['offer_completed_time'].values.max(axis=1)

In [54]:
print("number of offers in the final dataset:", offers.shape[0])
offers.head()

number of offers in the final dataset: 76277


,person,id_unique,offer_received_time,offer_viewed_time,offer_completed_time
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5-0.0,576.0,0.0,576.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed-0.0,336.0,372.0,0.0
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837-0.0,168.0,192.0,0.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d-0.0,408.0,456.0,414.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4-0.0,504.0,540.0,528.0


Note: 0s in offer_viewed_time and offer_completed_time mean they weren't viewed or completed

Now we need to add offer end time for each offer by adding offer valid time to its received time:

In [55]:
# add information about each offer from portfolio
offers['id'] = offers.id_unique.apply(lambda x: x.split("-")[0])
offers = offers.merge(portfolio, on='id', how='left')

In [56]:
offers['offer_end_time'] = offers['offer_received_time']+offers['duration'].values*24

In [57]:
offers.head()

,person,id_unique,offer_received_time,offer_viewed_time,offer_completed_time,id,reward,channels,difficulty,duration,offer_type,offer_end_time
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5-0.0,576.0,0.0,576.0,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,7,discount,744.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed-0.0,336.0,372.0,0.0,3f207df678b143eea3cee63160fa8bed,0,"[web, email, mobile]",0,4,informational,432.0
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837-0.0,168.0,192.0,0.0,5a8bc65990b245e5a138643cd4eb9837,0,"[email, mobile, social]",0,3,informational,240.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d-0.0,408.0,456.0,414.0,f19421c1d4aa40978ebb69ca19b0e20d,5,"[web, email, mobile, social]",5,5,bogo,528.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4-0.0,504.0,540.0,528.0,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,10,discount,744.0


Then I encode each offer that was viewed on time (i.e. if offer was viewed before its end time) with 1s, otherwise with 0s. The same procedure is repeated for offers completed on time. However, to be properly completed offer should have been viewed. Hence I create a new column "completed after viewing" to encode completed offers correctly.  

In [58]:
offers['viewed_binary'] = offers.offer_viewed_time.apply(lambda x: 1 if x > 0 else 0)

In [59]:
# 0 if viewed_time < end_time and 1 otherwise, multiply whether the offer was indeed viewed 
# (because 0 time is also < end time, but it means that the offer was not viewed!)
offers['viewed_on_time'] = (offers.offer_viewed_time < offers.offer_end_time)*offers['viewed_binary'] 

In [60]:
offers['completed_binary'] = offers.offer_completed_time.apply(lambda x: 1 if x > 0 else 0)

In [61]:
offers['completed_on_time'] = (offers.offer_completed_time < offers.offer_end_time)*offers['completed_binary'] 

In [62]:
# correctly attributed completed offers need to meet several conditions:
# 1. it should be completed before offer end
completed_before_expires = (offers.offer_completed_time < offers.offer_end_time)
# 2. it should be completed after viewing 
completed_after_viewing =(offers.offer_completed_time > offers.offer_viewed_time)*offers['viewed_binary']
offers['completed_after_viewing'] = (completed_after_viewing & completed_before_expires)*offers['completed_binary']

In [63]:
offers[offers.completed_after_viewing == 1].head()

,person,id_unique,offer_received_time,offer_viewed_time,offer_completed_time,id,reward,channels,difficulty,duration,offer_type,offer_end_time,viewed_binary,viewed_on_time,completed_binary,completed_on_time,completed_after_viewing
7,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7-0.0,408.0,432.0,576.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,"[web, email]",20,10,discount,648.0,1,1,1,1,1
8,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2-0.0,168.0,186.0,252.0,2298d6c36e964ae4a3e7e9706d1fb8c2,3,"[web, email, mobile, social]",7,7,discount,336.0,1,1,1,1,1
11,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9-0.0,504.0,516.0,576.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,"[web, email, mobile]",5,7,bogo,672.0,1,1,1,1,1
12,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0-0.0,408.0,426.0,510.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10,"[web, email, mobile, social]",10,5,bogo,528.0,1,1,1,1,1
15,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4-0.0,0.5,12.0,54.0,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,10,discount,240.5,1,1,1,1,1


Let's check one example:

In [64]:
offers[(offers.person == '0020c2b971eb4e9188eac86d93036a77') & (offers.id == 'fafdcd668e3743c1bb461111dcafc2a4')]

,person,id_unique,offer_received_time,offer_viewed_time,offer_completed_time,id,reward,channels,difficulty,duration,offer_type,offer_end_time,viewed_binary,viewed_on_time,completed_binary,completed_on_time,completed_after_viewing
15,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4-0.0,0.5,12.0,54.0,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,10,discount,240.5,1,1,1,1,1
16,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4-1.0,336.0,0.0,510.0,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,10,discount,576.0,0,0,1,1,0


In [65]:
starbucks[(starbucks.person == '0020c2b971eb4e9188eac86d93036a77') & (starbucks.offer_id == 'fafdcd668e3743c1bb461111dcafc2a4')]

,person,became_member_on,age,gender,income,time,offer_id,offer_duration,offer_reward,amount,rewarded,offer_completed,offer_received,offer_viewed,transaction,bogo,discount,informational
46131,0020c2b971eb4e9188eac86d93036a77,2016-03-04,59.0,F,90000.0,0,fafdcd668e3743c1bb461111dcafc2a4,10.0,2.0,NaN,NaN,0,1,0,0,0,1,0
46132,0020c2b971eb4e9188eac86d93036a77,2016-03-04,59.0,F,90000.0,12,fafdcd668e3743c1bb461111dcafc2a4,10.0,2.0,NaN,NaN,0,0,1,0,0,1,0
46134,0020c2b971eb4e9188eac86d93036a77,2016-03-04,59.0,F,90000.0,54,fafdcd668e3743c1bb461111dcafc2a4,10.0,2.0,NaN,2.0,1,0,0,0,0,1,0
46139,0020c2b971eb4e9188eac86d93036a77,2016-03-04,59.0,F,90000.0,336,fafdcd668e3743c1bb461111dcafc2a4,10.0,2.0,NaN,NaN,0,1,0,0,0,1,0
46144,0020c2b971eb4e9188eac86d93036a77,2016-03-04,59.0,F,90000.0,510,fafdcd668e3743c1bb461111dcafc2a4,10.0,2.0,NaN,2.0,1,0,0,0,0,1,0


In [66]:
print("total number of recorded offers completed:", offers.completed_binary.sum())
print("true number of offers completed after viewing and on time:", offers.completed_after_viewing.sum())
print("misattributed offers (false positives - completed, while not viewed; viewed after completed):", offers.completed_binary.sum()-offers.completed_after_viewing.sum())

total number of recorded offers completed: 33579
true number of offers completed after viewing and on time: 19656
misattributed offers (false positives - completed, while not viewed; viewed after completed): 13923


Let's summarize all the above steps in a function:

In [67]:
def aggregate_data_at_offers_level(starbucks):
    '''
    Aggregates data at the level of each offer with correctly attributed metrics 
    and information about each offer from portfolio dataset
    Input: 
        starbucks_merged: data frame that resulted from merge_clean_datasets()
    Output:
        offers: data frame aggregated at the offer level with correctly attributed metrics
    '''
    starbucks_time = starbucks[starbucks.transaction == 0][['person', 'offer_id', 'offer_received', 'offer_viewed', 'offer_completed', 'time']]
    
    # reencode time=0 with time=0.5 to avoid confusion with 0s after aggregation at later stage
    starbucks_time['time'] = starbucks_time.time.replace(0, 0.5)
    
    #create unique identifier for each offer sent (because same offers could be sent more than once)
    starbucks_time['id_unique_received'] = starbucks_time[starbucks_time.offer_received == 1].groupby(['person', 'offer_id']).cumcount()
    starbucks_time['id_unique_viewed'] = starbucks_time[starbucks_time.offer_viewed == 1].groupby(['person', 'offer_id']).cumcount()
    starbucks_time['id_unique_completed'] = starbucks_time[starbucks_time.offer_completed == 1].groupby(['person', 'offer_id']).cumcount()
    starbucks_time['id_unique_events'] = starbucks_time[['id_unique_received', 'id_unique_viewed', 'id_unique_completed']].max(axis=1).values
    starbucks_time['id_unique'] = starbucks_time['offer_id'] + "-" + starbucks_time['id_unique_events'].apply(lambda x: str(x))

    # create columns with time for each event
    starbucks_time['offer_received_time'] = starbucks_time['offer_received']*starbucks_time.time
    starbucks_time['offer_viewed_time'] = starbucks_time['offer_viewed']*starbucks_time.time
    starbucks_time['offer_completed_time'] = starbucks_time['offer_completed']*starbucks_time.time
    starbucks_time = starbucks_time[['person', 'id_unique', 'offer_id', 'time', 'offer_received_time', 'offer_viewed_time', 'offer_completed_time']]

    # unstack values to get to the level of each (person, offer id) tuple
    # need to take max value to avoid 0s
    starbucks_time_full = starbucks_time.groupby(by=['person', 'id_unique', 'time']).max().unstack()
    starbucks_time_full.fillna(0, inplace=True)

    # create the final time-based dataset for each offer sent
    offers = pd.DataFrame(starbucks_time_full.index.get_level_values('id_unique'), starbucks_time_full.index.get_level_values('person')).reset_index()
    offers['offer_received_time'] = starbucks_time_full['offer_received_time'].values.max(axis=1)
    offers['offer_viewed_time'] = starbucks_time_full['offer_viewed_time'].values.max(axis=1)
    offers['offer_completed_time'] = starbucks_time_full['offer_completed_time'].values.max(axis=1)

    # add information about each offer from portfolio
    offers['id'] = offers.id_unique.apply(lambda x: x.split("-")[0])
    offers = offers.merge(portfolio, on='id', how='left')

    # add offer end time
    offers['offer_end_time'] = offers['offer_received_time']+offers['duration'].values*24
        
    offers['viewed_binary'] = offers.offer_viewed_time.apply(lambda x: 1 if x > 0 else 0)
    # 0 if viewed_time < end_time and 1 otherwise, multiply whether the offer was indeed viewed 
    # (because 0 time is also < end time, but it means that the offer was not viewed!)
    offers['viewed_on_time'] = (offers.offer_viewed_time < offers.offer_end_time)*offers['viewed_binary'] 
    
    offers['completed_binary'] = offers.offer_completed_time.apply(lambda x: 1 if x > 0 else 0)
    offers['completed_on_time'] = (offers.offer_completed_time < offers.offer_end_time)*offers['completed_binary'] 
    
    # correctly attributed completed offers need to meet several conditions:
    # 1. it should be completed before offer end
    completed_before_expires = (offers.offer_completed_time < offers.offer_end_time)
    # 2. it should be completed after viewing 
    completed_after_viewing =(offers.offer_completed_time > offers.offer_viewed_time)*offers['viewed_binary']
    offers['completed_after_viewing'] = (completed_after_viewing & completed_before_expires)*offers['completed_binary']
    
    return offers

# Aggregation at person level

Now we are ready to aggreate this information on the customer level and count how many offers were received by each customer, how many offers were viewed on time, how many offers were completed on time (as recorded), how many offers were completed after viewing. 

In [68]:
offers_by_person = pd.DataFrame(offers.groupby('person').id_unique.count())
offers_by_person.shape

(16994, 1)

There are 6 people in the dataset that haven't received any offers. So our final dataset will be reduced by this number:

In [69]:
customers_without_offers = set(starbucks.person.unique()) - set(offers_by_person.reset_index().person.unique())

In [70]:
customers_without_offers

{'12ede229379747bd8d74ccdc20097ca3',
 '3a4874d8f0ef42b9a1b72294902afea9',
 'ae8111e7e8cd4b60a8d35c42c1110555',
 'c6e579c6821c41d1a7a6a9cf936e91bb',
 'da7a7c0dcfcb41a8acc7864a53cf60fb',
 'eb540099db834cf59001f83a4561aef3'}

In [71]:
offers_by_person.columns = ['offers_received']
offers_by_person['offers_viewed_on_time'] = pd.Series(offers.groupby('person').viewed_on_time.sum())
offers_by_person['offers_completed_on_time'] = pd.Series(offers.groupby('person').completed_on_time.sum())
offers_by_person['offers_completed_after_viewing'] = pd.Series(offers.groupby('person').completed_after_viewing.sum())

Finally, we can calculate the response rate (offers viewed/offers received) and conversion rates (offers completed after viewing/offers viewed) for each customer:

In [72]:
offers_by_person['offers_rr'] = offers_by_person.offers_viewed_on_time/offers_by_person.offers_received
offers_by_person['offers_cvr'] = offers_by_person.offers_completed_after_viewing/offers_by_person.offers_viewed_on_time

In [73]:
offers.shape

(76277, 17)

In [74]:
offers_by_person.head()

,offers_received,offers_viewed_on_time,offers_completed_on_time,offers_completed_after_viewing,offers_rr,offers_cvr
person,,,,,,
0009655768c64bdeb2e877511632db8f,5,4,3,0,0.8,0.00
00116118485d4dfda04fdbaba9a87b5c,2,2,0,0,1.0,0.00
0011e0d4e6b944f998e987f904e8c1e5,5,5,3,3,1.0,0.60
0020c2b971eb4e9188eac86d93036a77,5,2,3,2,0.4,1.00
0020ccbbb6d84e358d3414a3ff76cffd,4,4,3,3,1.0,0.75


Now let's calculate correctly and incorrectly attributed rewards per person:

In [75]:
# CORRECTLY attributed BOGO amount (completed after viewing)
cond_correct = (offers.offer_type == 'bogo') & (offers.completed_after_viewing == 1) 
total_bogo = offers[cond_correct].groupby(by='person').reward.sum()
offers_by_person = offers_by_person.merge(total_bogo, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_bogo'}, inplace=True)

# CORRECTLY attributed BOGO number (completed after viewing)
bogo = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(bogo, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_completed'}, inplace=True)

In [76]:
# INCORRECTLY attributed BOGO amount 
cond_incorrect = ((offers.offer_type == 'bogo') & (offers.completed_after_viewing == 0) & (offers.completed_binary == 1))
total_bogo_incorr = offers[cond_incorrect].groupby(by='person').reward.sum()
offers_by_person = offers_by_person.merge(total_bogo_incorr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_bogo_incorr'}, inplace=True)

# INCORRECTLY attributed BOGO number (completed without viewing)
bogo_incorr = offers[cond_incorrect].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(bogo_incorr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_completed_incorr'}, inplace=True)

In [77]:
print("number of customers with bogo completed (as recorded):", ((offers.offer_type == 'bogo') & (offers.completed_binary == 1)).sum())
print("number of customers with correctly attributed completed bogo offers:", offers[cond_correct].shape[0])
print("number of customers with incorrectly attributed completed bogo offers:", offers[cond_incorrect].shape[0])

number of customers with bogo completed (as recorded): 15669
number of customers with correctly attributed completed bogo offers: 8843
number of customers with incorrectly attributed completed bogo offers: 6826


In [78]:
# CORRECTLY attributed DISCOUNT amount (completed after viewing)
cond_correct = (offers.offer_type == 'discount') & (offers.completed_after_viewing == 1) 
total_bogo_corr = offers[cond_correct].groupby(by='person').reward.sum()
offers_by_person = offers_by_person.merge(total_bogo_corr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_discount'}, inplace=True)

# CORRECTLY attributed DISCOUNT number (completed after viewing)
discount = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(discount, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_completed'}, inplace=True)

In [79]:
# INCORRECTLY attributed DISCOUNT amount 
cond_incorrect = ((offers.offer_type == 'discount') & (offers.completed_after_viewing == 0) & (offers.completed_binary == 1))
total_bogo_incorr = offers[cond_incorrect].groupby(by='person').reward.sum()
offers_by_person = offers_by_person.merge(total_bogo_incorr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_discount_incorr'}, inplace=True)

# INCORRECTLY attributed DISCOUNT number (completed without viewing)
discount_incorr = offers[cond_incorrect].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(discount_incorr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_completed_incorr'}, inplace=True)

In [80]:
print("number of discounts completed (as recorded):", ((offers.offer_type == 'discount') & (offers.completed_binary == 1)).sum())
print("number of correctly attributed completed discount offers:", offers[cond_correct].shape[0])
print("number of incorrectly attributed completed discount offers:", offers[cond_incorrect].shape[0])

number of discounts completed (as recorded): 17910
number of correctly attributed completed discount offers: 10813
number of incorrectly attributed completed discount offers: 7097


In [81]:
# CORRECTLY attributed INFORMATIONAL number (viewed on time)
cond_correct = ((offers.offer_type == 'informational') & (offers.viewed_on_time == 1))
                                                              
informational = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(informational, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'informational_viewed'}, inplace=True)

In [82]:
# INCORRECTLY attributed INFORMATIONAL number (viewed not on time)
cond_incorrect = ((offers.offer_type == 'informational') & (offers.viewed_on_time == 0))
                                                              
informational_incorr = offers[cond_incorrect].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(informational_incorr, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'informational_incorr'}, inplace=True)

In [83]:
offers_by_person.head(10)

,offers_received,offers_viewed_on_time,offers_completed_on_time,offers_completed_after_viewing,offers_rr,offers_cvr,total_bogo,bogo_completed,total_bogo_incorr,bogo_completed_incorr,total_discount,discount_completed,total_discount_incorr,discount_completed_incorr,informational_viewed,informational_incorr
person,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5,4,3,0,0.800000,0.00,NaN,NaN,5.0,1.0,NaN,NaN,4.0,2.0,2.0,NaN
00116118485d4dfda04fdbaba9a87b5c,2,2,0,0,1.000000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011e0d4e6b944f998e987f904e8c1e5,5,5,3,3,1.000000,0.60,5.0,1.0,NaN,NaN,8.0,2.0,NaN,NaN,2.0,NaN
0020c2b971eb4e9188eac86d93036a77,5,2,3,2,0.400000,1.00,10.0,1.0,NaN,NaN,2.0,1.0,2.0,1.0,NaN,1.0
0020ccbbb6d84e358d3414a3ff76cffd,4,4,3,3,1.000000,0.75,10.0,2.0,NaN,NaN,3.0,1.0,NaN,NaN,1.0,NaN
003d66b6608740288d6cc97a6903f4f0,5,4,3,2,0.800000,0.50,NaN,NaN,NaN,NaN,4.0,2.0,5.0,1.0,2.0,NaN
00426fe3ffde4c6b9cb9ad6d077a13ea,5,2,1,1,0.400000,0.50,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN
004b041fbfe44859945daa2c7f79ee64,3,2,2,2,0.666667,1.00,5.0,1.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,1.0
004c5799adbf42868b9cff0396190900,5,4,5,3,0.800000,0.75,10.0,2.0,10.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN


In [84]:
print("total number of recorded offers completed:", offers.completed_binary.sum())
print("true number of offers completed after viewing and on time:", offers.completed_after_viewing.sum())
print("misattributed offers (false positives - completed, while not viewed; viewed after completed):", offers.completed_binary.sum()-offers.completed_after_viewing.sum())
print(70*"-")
print("BOGO:")
print("number of customers with bogo completed (as recorded):", ((offers.offer_type == 'bogo') & (offers.completed_binary == 1)).sum())
print("number of correctly attributed completed bogo offers:", offers_by_person.bogo_completed.sum())
print("number of incorrectly attributed completed bogo offers:", offers_by_person.bogo_completed_incorr.sum())
print(70*"-")
print("DISCOUNT:")
print("number of customers with discounts completed (as recorded):", ((offers.offer_type == 'discount') & (offers.completed_binary == 1)).sum())
print("number of correctly attributed completed discount offers:", offers_by_person.discount_completed.sum())
print("number of incorrectly attributed completed discount offers:", offers_by_person.discount_completed_incorr.sum())

total number of recorded offers completed: 33579
true number of offers completed after viewing and on time: 19656
misattributed offers (false positives - completed, while not viewed; viewed after completed): 13923
----------------------------------------------------------------------
BOGO:
number of customers with bogo completed (as recorded): 15669
number of correctly attributed completed bogo offers: 8843.0
number of incorrectly attributed completed bogo offers: 6826.0
----------------------------------------------------------------------
DISCOUNT:
number of customers with discounts completed (as recorded): 17910
number of correctly attributed completed discount offers: 10813.0
number of incorrectly attributed completed discount offers: 7097.0


In [85]:
# total amount rewarded
offers_by_person['total_rewarded'] = pd.DataFrame(starbucks.groupby('person').rewarded.sum())
cond_corr = (offers.completed_after_viewing == 1)
offers_by_person['total_rewarded_corr'] = pd.DataFrame(offers[cond_corr].groupby('person').reward.sum())
cond_incorr = (offers.completed_after_viewing == 0) & (offers.completed_binary == 1)
offers_by_person['total_rewarded_incorr'] = pd.DataFrame(offers[cond_incorr].groupby('person').reward.sum())

In [86]:
print(f"total amount rewarded: ${offers_by_person.total_rewarded.sum():.0f}")
print(f"total amount correctly rewarded: ${offers_by_person.total_rewarded_corr.sum():.0f}")
print(f"total amount wasted (incorrectly rewarded): ${offers_by_person.total_rewarded_incorr.sum():.0f}")
print(70*"-")
print(f"BOGO rewarded correctly: ${offers_by_person.total_bogo.sum():.0f}")
print(f"BOGO wasted: ${offers_by_person.total_bogo_incorr.sum():.0f}")
print(70*"-")
print(f"DISCOUNT rewarded correctly: ${offers_by_person.total_discount.sum():.0f}")
print(f"DISCOUNT wasted:${offers_by_person.total_discount_incorr.sum():.0f}")

total amount rewarded: $164676
total amount correctly rewarded: $94893
total amount wasted (incorrectly rewarded): $69783
----------------------------------------------------------------------
BOGO rewarded correctly: $66005
BOGO wasted: $47435
----------------------------------------------------------------------
DISCOUNT rewarded correctly: $28888
DISCOUNT wasted:$22348


Adding bogo_rr, bogo_cvr:

In [87]:
# CORRECTLY attributed BOGO viewed number 
cond_correct = (offers.offer_type == 'bogo') & (offers.viewed_on_time == 1) 

bogo_viewed = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(bogo_viewed, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_viewed'}, inplace=True)

In [88]:
# BOGO received number 
cond_correct = (offers.offer_type == 'bogo')

bogo_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(bogo_received, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_received'}, inplace=True)

In [89]:
offers_by_person['bogo_rr'] = offers_by_person.bogo_viewed/offers_by_person.bogo_received
offers_by_person['bogo_cvr'] = offers_by_person.bogo_completed/offers_by_person.bogo_viewed

Addin discount_rr, discount_cvr:

In [90]:
# CORRECTLY attributed DISCOUNT viewed number 
cond_correct = (offers.offer_type == 'discount') & (offers.viewed_on_time == 1) 

discount_viewed = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(discount_viewed, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_viewed'}, inplace=True)

In [91]:
# DISCOUNT received number 
cond_correct = (offers.offer_type == 'discount')

discount_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(discount_received, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_received'}, inplace=True)

In [92]:
offers_by_person['discount_rr'] = offers_by_person.discount_viewed/offers_by_person.discount_received
offers_by_person['discount_cvr'] = offers_by_person.discount_completed/offers_by_person.discount_viewed

Adding informational_rr:

In [93]:
# INFORMATIONAL received number 
cond_correct = (offers.offer_type == 'informational')

informational_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
offers_by_person = offers_by_person.merge(informational_received, on='person', how='left')
offers_by_person.rename(columns={offers_by_person.columns[-1]:'informational_received'}, inplace=True)

In [94]:
offers_by_person['informational_rr'] = offers_by_person.informational_viewed/offers_by_person.informational_received

In [95]:
offers_by_person.columns

Index(['offers_received', 'offers_viewed_on_time', 'offers_completed_on_time',
       'offers_completed_after_viewing', 'offers_rr', 'offers_cvr',
       'total_bogo', 'bogo_completed', 'total_bogo_incorr',
       'bogo_completed_incorr', 'total_discount', 'discount_completed',
       'total_discount_incorr', 'discount_completed_incorr',
       'informational_viewed', 'informational_incorr', 'total_rewarded',
       'total_rewarded_corr', 'total_rewarded_incorr', 'bogo_viewed',
       'bogo_received', 'bogo_rr', 'bogo_cvr', 'discount_viewed',
       'discount_received', 'discount_rr', 'discount_cvr',
       'informational_received', 'informational_rr'],
      dtype='object')

In [96]:
offers_by_person_fin = offers_by_person[['offers_received', 'offers_viewed_on_time','offers_completed_after_viewing',
                                        'offers_rr', 'offers_cvr', 
                                        'bogo_rr', 'bogo_cvr', 
                                        'discount_rr', 'discount_cvr',
                                        'informational_rr',
                                        'discount_received', 'discount_viewed','discount_completed',
                                        'bogo_received', 'bogo_viewed', 'bogo_completed',
                                        'total_discount','total_bogo', 'total_rewarded_corr',
                                        'informational_viewed', 'informational_received']]

In [97]:
offers_by_person_fin = offers_by_person_fin.fillna(0)

Add information from profile:

In [98]:
starbucks_aggr = profile_clean.merge(offers_by_person, on='person', how='left')

# fill NaNs in all columns except age, income, gender with 0s for 6 people without offers
cols_notprofile = starbucks_aggr.columns.difference(['age', 'income', 'gender'])
starbucks_aggr[cols_notprofile] = starbucks_aggr[cols_notprofile].fillna(0)

Note: I added back 6 people that didn't receive any offers and filled all columns generated based on offer interactions with 0s. Otherwise, run the first line without how='left' and don't add fillna()

In [99]:
starbucks_aggr.shape

(17000, 34)

Add information on transactions amount and transactions_num:

In [100]:
spending = starbucks.groupby('person')[['amount', 'transaction']].sum()

In [101]:
starbucks_aggr = starbucks_aggr.merge(spending, on='person', how='left')
starbucks_aggr.rename(columns={starbucks_aggr.columns[-1]:'transactions_num'}, inplace=True)
starbucks_aggr.rename(columns={starbucks_aggr.columns[-2]:'total_amount'}, inplace=True)

In [102]:
starbucks_aggr.head()

,person,became_member_on,gender,age,income,offers_received,offers_viewed_on_time,offers_completed_on_time,offers_completed_after_viewing,offers_rr,...,bogo_rr,bogo_cvr,discount_viewed,discount_received,discount_rr,discount_cvr,informational_received,informational_rr,total_amount,transactions_num
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,NaN,NaN,5.0,5.0,2.0,2.0,1.0,...,0.0,0.000000,5.0,5.0,1.000000,0.4,0.0,0.0,20.40,9
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55.0,112000.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,77.01,3
2,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.5,...,1.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,14.30,6
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75.0,100000.0,4.0,4.0,3.0,2.0,1.0,...,1.0,0.666667,0.0,0.0,0.000000,0.0,1.0,1.0,159.27,7
4,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,NaN,NaN,5.0,3.0,0.0,0.0,0.6,...,0.0,0.000000,2.0,3.0,0.666667,0.0,2.0,0.5,4.65,3


In [103]:
starbucks_aggr = starbucks_aggr[['gender', 'age', 'person', 'became_member_on', 'income', 'total_amount',
                                 'total_rewarded_corr', 'transactions_num', 'offers_received', 
                                 'offers_viewed_on_time', 'offers_completed_after_viewing',
                                'bogo_received', 'bogo_viewed', 'bogo_completed',
                                'discount_received', 'discount_viewed', 'discount_completed',
                                'informational_received', 'informational_viewed',
                                'total_bogo', 'total_discount']]

In [104]:
starbucks_aggr.columns = ['gender', 'age', 'person', 'became_member_on', 'income', 'total_amount',
                          'total_rewarded', 'transactions_num', 'offers_received', 
                          'offers_viewed', 'offers_completed',
                          'bogo_received', 'bogo_viewed', 'bogo_completed',
                          'discount_received', 'discount_viewed', 'discount_completed',
                          'informational_received', 'informational_viewed',
                          'total_bogo', 'total_discount']

In [105]:
def aggregate_data_at_person_level(offers):
    '''
    Aggregates data at person level
    Input: 
        offers: data frame resulted from calling aggregate_data_at_offers_level()
    Output:
        starbucks_aggr: data frame with aggregated data at person level and correctly attributed metrics
    '''
    offers_by_person = pd.DataFrame(offers.groupby('person').id_unique.count())
    offers_by_person.columns = ['offers_received']
    offers_by_person['offers_viewed'] = pd.Series(offers.groupby('person').viewed_on_time.sum())
    offers_by_person['offers_completed'] = pd.Series(offers.groupby('person').completed_after_viewing.sum())
    offers_by_person['offers_rr'] = offers_by_person.offers_viewed/offers_by_person.offers_received
    offers_by_person['offers_cvr'] = offers_by_person.offers_completed/offers_by_person.offers_viewed
    
    # BOGO received number 
    cond_correct = (offers.offer_type == 'bogo')
    bogo_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(bogo_received, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_received'}, inplace=True)
    
    # CORRECTLY attributed BOGO viewed number 
    cond_correct = (offers.offer_type == 'bogo') & (offers.viewed_on_time == 1) 
    bogo_viewed = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(bogo_viewed, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_viewed'}, inplace=True)

    # CORRECTLY attributed BOGO amount (completed after viewing)
    cond_correct = (offers.offer_type == 'bogo') & (offers.completed_after_viewing == 1) 
    total_bogo = offers[cond_correct].groupby(by='person').reward.sum()
    offers_by_person = offers_by_person.merge(total_bogo, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_bogo'}, inplace=True)

    # CORRECTLY attributed BOGO comleted number (completed after viewing)
    bogo = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(bogo, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'bogo_completed'}, inplace=True)
    
    # DISCOUNT received number 
    cond_correct = (offers.offer_type == 'discount')
    discount_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(discount_received, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_received'}, inplace=True)

    # CORRECTLY attributed DISCOUNT viewed number 
    cond_correct = (offers.offer_type == 'discount') & (offers.viewed_on_time == 1) 
    discount_viewed = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(discount_viewed, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_viewed'}, inplace=True)

    # CORRECTLY attributed DISCOUNT amount (completed after viewing)
    cond_correct = (offers.offer_type == 'discount') & (offers.completed_after_viewing == 1) 
    total_bogo_corr = offers[cond_correct].groupby(by='person').reward.sum()
    offers_by_person = offers_by_person.merge(total_bogo_corr, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'total_discount'}, inplace=True)

    # CORRECTLY attributed DISCOUNT completed number (completed after viewing)
    discount = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(discount, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'discount_completed'}, inplace=True)
    
    # INFORMATIONAL received number 
    cond_correct = (offers.offer_type == 'informational')

    informational_received = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(informational_received, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'informational_received'}, inplace=True)

    # CORRECTLY attributed INFORMATIONAL viewed number (viewed on time)
    cond_correct = ((offers.offer_type == 'informational') & (offers.viewed_on_time == 1))

    informational = offers[cond_correct].groupby(by='person')['id_unique'].count()
    offers_by_person = offers_by_person.merge(informational, on='person', how='left')
    offers_by_person.rename(columns={offers_by_person.columns[-1]:'informational_viewed'}, inplace=True)
    
    # CORRECT AMOUNT REWARDED
    cond_corr = (offers.completed_after_viewing == 1)
    offers_by_person['total_rewarded'] = pd.DataFrame(offers[cond_corr].groupby('person').reward.sum())

    offers_by_person['bogo_rr'] = offers_by_person.bogo_viewed/offers_by_person.bogo_received
    offers_by_person['bogo_cvr'] = offers_by_person.bogo_completed/offers_by_person.bogo_viewed
    offers_by_person['discount_rr'] = offers_by_person.discount_viewed/offers_by_person.discount_received
    offers_by_person['discount_cvr'] = offers_by_person.discount_completed/offers_by_person.discount_viewed
    offers_by_person['informational_rr'] = offers_by_person.informational_viewed/offers_by_person.informational_received
    
    offers_by_person = offers_by_person.fillna(0)
    
    #merge with profile info
    profile_clean = clean_profile_dataset(profile)
    starbucks_aggr = profile_clean.merge(offers_by_person, on='person', how='left')
    
    # fill NaNs in all columns except age, income, gender with 0s for 6 people without offers
    cols_notprofile = starbucks_aggr.columns.difference(['age', 'income', 'gender'])
    starbucks_aggr[cols_notprofile] = starbucks_aggr[cols_notprofile].fillna(0)
    
    #add spending information
    spending = starbucks.groupby('person')[['amount', 'transaction']].sum()
    starbucks_aggr = starbucks_aggr.merge(spending, on='person', how='left')
    starbucks_aggr.rename(columns={starbucks_aggr.columns[-1]:'transactions_num'}, inplace=True)
    starbucks_aggr.rename(columns={starbucks_aggr.columns[-2]:'total_amount'}, inplace=True)
    
    #reorder columns
    starbucks_aggr = starbucks_aggr[['gender', 'age', 'person', 'became_member_on', 'income', 'total_amount',
                                 'total_rewarded', 'transactions_num', 'offers_received', 
                                 'offers_viewed', 'offers_completed',
                                'bogo_received', 'bogo_viewed', 'bogo_completed',
                                'discount_received', 'discount_viewed', 'discount_completed',
                                'informational_received', 'informational_viewed',
                                'total_bogo', 'total_discount']]
    return starbucks_aggr

In [106]:
offers = aggregate_data_at_offers_level(starbucks)
starbucks_aggr = aggregate_data_at_person_level(offers)

In [107]:
starbucks_aggr.shape

(17000, 21)

In [108]:
starbucks_aggr.head()

,gender,age,person,became_member_on,income,total_amount,total_rewarded,transactions_num,offers_received,offers_viewed,...,bogo_received,bogo_viewed,bogo_completed,discount_received,discount_viewed,discount_completed,informational_received,informational_viewed,total_bogo,total_discount
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,20.40,5.0,9,5.0,5.0,...,0.0,0.0,0.0,5.0,5.0,2.0,0.0,0.0,0.0,5.0
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,77.01,0.0,3,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,14.30,0.0,6,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,159.27,15.0,7,4.0,4.0,...,3.0,3.0,2.0,0.0,0.0,0.0,1.0,1.0,15.0,0.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,4.65,0.0,3,5.0,3.0,...,0.0,0.0,0.0,3.0,2.0,0.0,2.0,1.0,0.0,0.0


In [109]:
starbucks_aggr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17000 entries, 0 to 16999
Data columns (total 21 columns):
gender                    14825 non-null object
age                       14825 non-null float64
person                    17000 non-null object
became_member_on          17000 non-null datetime64[ns]
income                    14825 non-null float64
total_amount              17000 non-null float64
total_rewarded            17000 non-null float64
transactions_num          17000 non-null uint8
offers_received           17000 non-null float64
offers_viewed             17000 non-null float64
offers_completed          17000 non-null float64
bogo_received             17000 non-null float64
bogo_viewed               17000 non-null float64
bogo_completed            17000 non-null float64
discount_received         17000 non-null float64
discount_viewed           17000 non-null float64
discount_completed        17000 non-null float64
informational_received    17000 non-null float64
inform

## Feature Engineering

In [110]:
starbucks_engineered = starbucks_aggr.copy()

In [111]:
# reencode became_member_on as membership by years in str format (categorical)
starbucks_engineered['became_member_on'] = starbucks_engineered.became_member_on.apply(lambda x: str(x.year))

In [112]:
# create 4 new columns:
starbucks_engineered['avg_order_size'] = round(starbucks_engineered['total_amount']/starbucks_engineered['transactions_num'], 2)
# when there were no transactions, avg_order_size was converted to NaNs, replace these with 0:
starbucks_engineered['avg_order_size'] = starbucks_engineered['avg_order_size'].replace(np.nan, 0)

starbucks_engineered['avg_reward_size'] = round(starbucks_engineered['total_rewarded']/starbucks_engineered['offers_completed'], 2)
# when there were no offers completed, avg_reward_size was converted to NaNs, replace these with 0:
starbucks_engineered['avg_reward_size'] = starbucks_engineered['avg_reward_size'].replace(np.nan, 0)

# average bogo size and average discount size
for each in ['bogo', 'discount']:
    starbucks_engineered['avg_'+each+'_size'] = round(starbucks_engineered['total_'+each]/starbucks_engineered[each+'_completed'], 2)
    starbucks_engineered['avg_'+each+'_size'] = starbucks_engineered['avg_'+each+'_size'].replace(np.nan, 0)

In [113]:
# create new columns with CVR and RR for each offer type
for each in ['offers', 'bogo', 'discount', 'informational']:
    # reponse rate, % offers viewed to offers received
    starbucks_engineered[each+'_rr'] = round(starbucks_engineered[each+'_viewed']/starbucks_engineered[each+'_received'] * 100, 2)
    # replace div by 0 - if no offers received, then rr = 0
    starbucks_engineered[each+'_rr'] = starbucks_engineered[each+'_rr'].replace(np.nan, 0).replace(np.inf, 0)
    
    # conversion rate, % offers completed to offers viewed
    if each != 'informational':
        starbucks_engineered[each+'_cvr'] = round(starbucks_engineered[each+'_completed']/starbucks_engineered[each+'_viewed'] * 100, 2)
        # replace div by 0 - if no offers viewed, then cvr = 0
        starbucks_engineered[each+'_cvr'] = starbucks_engineered[each+'_cvr'].replace(np.inf, 0).replace(np.nan, 0)

In [114]:
starbucks_engineered.head()

,gender,age,person,became_member_on,income,total_amount,total_rewarded,transactions_num,offers_received,offers_viewed,...,avg_reward_size,avg_bogo_size,avg_discount_size,offers_rr,offers_cvr,bogo_rr,bogo_cvr,discount_rr,discount_cvr,informational_rr
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017,NaN,20.40,5.0,9,5.0,5.0,...,2.5,0.0,2.5,100.0,40.0,0.0,0.00,100.00,40.0,0.0
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017,112000.0,77.01,0.0,3,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018,NaN,14.30,0.0,6,2.0,1.0,...,0.0,0.0,0.0,50.0,0.0,100.0,0.00,0.00,0.0,0.0
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017,100000.0,159.27,15.0,7,4.0,4.0,...,7.5,7.5,0.0,100.0,50.0,100.0,66.67,0.00,0.0,100.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017,NaN,4.65,0.0,3,5.0,3.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.00,66.67,0.0,50.0


Let's summarize above steps in a function:

In [115]:
def engineer_new_features(starbucks_aggr):
    '''
    Engineers new features based on data that we already have
    Input:
        starbucks_aggr: data frame resulted from calling aggregate_data_at_person_level()
    Output:
        starbucks_engineered: the same data frame with new features added
    '''
    starbucks_engineered = starbucks_aggr.copy()
    
    # reencode became_member_on as membership by years in str format (categorical)
    starbucks_engineered['became_member_on'] = starbucks_engineered.became_member_on.apply(lambda x: str(x.year))    
    
    # create 4 new columns:
    starbucks_engineered['avg_order_size'] = round(starbucks_engineered['total_amount']/starbucks_engineered['transactions_num'], 2)
    # when there were no transactions, avg_order_size was converted to NaNs, replace these with 0:
    starbucks_engineered['avg_order_size'] = starbucks_engineered['avg_order_size'].replace(np.nan, 0)

    starbucks_engineered['avg_reward_size'] = round(starbucks_engineered['total_rewarded']/starbucks_engineered['offers_completed'], 2)
    # when there were no offers completed, avg_reward_size was converted to NaNs, replace these with 0:
    starbucks_engineered['avg_reward_size'] = starbucks_engineered['avg_reward_size'].replace(np.nan, 0)

    # average bogo size and average discount size
    for each in ['bogo', 'discount']:
        starbucks_engineered['avg_'+each+'_size'] = round(starbucks_engineered['total_'+each]/starbucks_engineered[each+'_completed'], 2)
        starbucks_engineered['avg_'+each+'_size'] = starbucks_engineered['avg_'+each+'_size'].replace(np.nan, 0)        
    
    # create new columns with CVR and RR for each offer type
    for each in ['offers', 'bogo', 'discount', 'informational']:
        # reponse rate, % offers viewed to offers received
        starbucks_engineered[each+'_rr'] = round(starbucks_engineered[each+'_viewed']/starbucks_engineered[each+'_received'] * 100, 2)
        # replace div by 0 - if no offers received, then rr = 0
        starbucks_engineered[each+'_rr'] = starbucks_engineered[each+'_rr'].replace(np.nan, 0).replace(np.inf, 0)

        # conversion rate, % offers completed to offers viewed
        if each != 'informational':
            starbucks_engineered[each+'_cvr'] = round(starbucks_engineered[each+'_completed']/starbucks_engineered[each+'_viewed'] * 100, 2)
            # replace div by 0 - if no offers viewed, then cvr = 0
            starbucks_engineered[each+'_cvr'] = starbucks_engineered[each+'_cvr'].replace(np.inf, 0).replace(np.nan, 0)        
    
    return starbucks_engineered

In [116]:
starbucks_engineered = engineer_new_features(starbucks_aggr)

In [117]:
starbucks_engineered.head()

,gender,age,person,became_member_on,income,total_amount,total_rewarded,transactions_num,offers_received,offers_viewed,...,avg_reward_size,avg_bogo_size,avg_discount_size,offers_rr,offers_cvr,bogo_rr,bogo_cvr,discount_rr,discount_cvr,informational_rr
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017,NaN,20.40,5.0,9,5.0,5.0,...,2.5,0.0,2.5,100.0,40.0,0.0,0.00,100.00,40.0,0.0
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017,112000.0,77.01,0.0,3,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018,NaN,14.30,0.0,6,2.0,1.0,...,0.0,0.0,0.0,50.0,0.0,100.0,0.00,0.00,0.0,0.0
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017,100000.0,159.27,15.0,7,4.0,4.0,...,7.5,7.5,0.0,100.0,50.0,100.0,66.67,0.00,0.0,100.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017,NaN,4.65,0.0,3,5.0,3.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.00,66.67,0.0,50.0


This is our final dataset that we will be working with. Let's save it into csv file.

In [118]:
starbucks_engineered.to_csv(path_or_buf='data/starbucks_customer_level.csv', sep=';', index=False)

In [119]:
offers.to_csv(path_or_buf='data/starbucks_offer_level.csv', sep=';', index=False)